# Hello Overture

Some tests to expore the overture-maps data and see what we can do with it.

## Prerequisites

Follow README.md to download data and setup an Python environment using conda.

## Setup

import and load dependencies

In [1]:
import duckdb
import json
import pandas as pd
import geopandas as gpd

from keplergl import KeplerGl

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

# Set configurations on jupysql to directly output data to Pandas and to simplify the output that is printed to the notebook.
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Set configurations on jupysql to directly output data to Pandas and to simplify the output that is printed to the notebook.
%sql duckdb:///:default:
    
# Load spatial extension for DuckDB
%sql INSTALL spatial
%sql LOAD spatial


,Success


## Functions

### Set local and english names

In [2]:
def append_local_and_en_names(row):
    names = json.loads(row.names)['common'] if pd.notna(row['names']) else []

    for name in names:
        if name['language'] == 'local':
            row['localname'] = name['value']

        if name['language'] in ['en', 'en-Latn']:
            row['enname'] = name['value']

    return row

## Get some data and show it on a map

### Select country (NL)

In [3]:
%%sql selected_country << 
    SELECT
        isoCountryCodeAlpha2,
        JSON(names) as names,
        JSON(sources) as sources,
        geometry as geom
    FROM read_parquet('../data/overture_maps/theme=admins/type=*/*', hive_partitioning=1)
    WHERE adminLevel = 2
    AND isoCountryCodeAlpha2 = 'NL'
    

Set local and english names

In [4]:
selected_country = selected_country.apply(append_local_and_en_names, axis=1)

### Select provinces within country

In [5]:
%%sql selected_provinces << 
    SELECT
        localityType,
        JSON(a.names) as names,
        JSON(a.sources) as sources,
        a.geometry as geom,
        a.isosubcountrycode
    FROM read_parquet('../data/overture_maps/theme=admins/type=*/*', hive_partitioning=1) as a, selected_country as b
    WHERE adminLevel = 4
    AND localityType = 'province'
    AND ST_Within(ST_GeomFromWKB(a.geometry), ST_GeomFromWKB(b.geom))
    

Get the names in new columns and remove unused data

In [6]:
selected_provinces = selected_provinces.apply(append_local_and_en_names, axis=1)
selected_provinces = selected_provinces.drop(columns=["names", "sources"])

### Get buildings within bounds

In [7]:
den_bosch_bbox = ["POLYGON ((5.300171258675078 51.69073045148227, 5.300611111690557 51.68445866410491, 5.315566114235423 51.68486772020245, 5.313366849155244 51.690866785036064, 5.300171258675078 51.69073045148227))"]
data_bbox = pd.DataFrame(den_bosch_bbox, columns=['bbox'])
#https://dev.to/savo/spatial-data-analysis-with-duckdb-40j9

In [8]:
%%sql selected_buildings << 
    SELECT
        height,
        class,
        numFloors,
        a.geometry as geom
    FROM read_parquet('../data/overture_maps/theme=buildings/type=building/*', hive_partitioning=1) as a, data_bbox as b
    WHERE ST_Intersects(ST_GeomFromWKB(a.geometry), ST_GeomFromText(b.bbox))

In [9]:
selected_buildings['height'] = selected_buildings['height'].fillna(0)
selected_buildings['numfloors'] = selected_buildings['numfloors'].fillna(1)
selected_buildings['class'] = selected_buildings['class'].fillna('')

### Get roads within bounds

only subtype road is currently available in the data

In [10]:
%%sql selected_roads << 
    SELECT
        replace(json_extract(road::json,'$.class'),'"','')::varchar as class,
        replace(json_extract(road::json,'$.surface'),'"','')::varchar as surface,
        replace(json_extract(road::json,'$.roadNames.common[0].value'),'"','')::varchar as roadName,
        replace(json_extract(road::json,'$.restrictions.speedLimits[0].maxSpeed'),'"','')::varchar as speedLimit,
        geometry as geom
    FROM read_parquet('../data/overture_maps/theme=transportation/type=segment/*', hive_partitioning=1) as a, data_bbox as b
    WHERE subType = 'road'
    AND ST_Intersects(ST_GeomFromWKB(a.geometry), ST_GeomFromText(b.bbox))

In [11]:
selected_roads['surface'] = selected_roads['surface'].fillna('unknown')
selected_roads['roadName'] = selected_roads['roadName'].fillna('unknown')
selected_roads['speedLimit'] = selected_roads['speedLimit'].fillna(-1)

### Get places within bounds

In [12]:
%%sql selected_places << 
    SELECT
        replace(json_extract(addresses::json,'$[0].locality'),'"','')::varchar as locality,
        replace(json_extract(addresses::json,'$[0].region'),'"','')::varchar as region,
        replace(json_extract(addresses::json,'$[0].postcode'),'"','')::varchar as postcode,
        replace(json_extract(addresses::json,'$[0].freeform'),'"','')::varchar as freeform,
        replace(json_extract(categories::json,'$.main'),'"','')::varchar as categories_main,
        replace(json_extract(names::json,'$.common[0].value'),'"','')::varchar as names,
        confidence,
        geometry as geom
    FROM read_parquet('../data/overture_maps/theme=places/type=place/*', hive_partitioning=1) as a, data_bbox as b
    WHERE ST_Intersects(ST_GeomFromWKB(a.geometry), ST_GeomFromText(b.bbox))
    AND confidence > 0.75

In [13]:
selected_places = selected_places.fillna('')

## Show data on map

In [14]:
country = gpd.GeoDataFrame(selected_country, geometry=gpd.GeoSeries.from_wkb(selected_country.geom.apply(bytes)))
provinces = gpd.GeoDataFrame(selected_provinces, geometry=gpd.GeoSeries.from_wkb(selected_provinces.geom.apply(bytes)))
buildings = gpd.GeoDataFrame(selected_buildings, geometry=gpd.GeoSeries.from_wkb(selected_buildings.geom.apply(bytes)))
roads = gpd.GeoDataFrame(selected_roads, geometry=gpd.GeoSeries.from_wkb(selected_roads.geom.apply(bytes)))
places = gpd.GeoDataFrame(selected_places, geometry=gpd.GeoSeries.from_wkb(selected_places.geom.apply(bytes)))

places['Longitude'] = places.geometry.apply(lambda geom: geom.x)
places['Latitude'] = places.geometry.apply(lambda geom: geom.y)

with open("./map_configs/hello_overture_map_1.json", "r") as f:
    map_config = f.read()
    
map_1 = KeplerGl(config=json.loads(map_config), height=800, show_docs=False)
map_1.add_data(data=places, name='places')
map_1.add_data(data=roads, name='roads')
map_1.add_data(data=buildings, name='buildings')
map_1.add_data(data=provinces, name='provinces')
map_1.add_data(data=country, name='country')
map_1


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'our7thl', 'type': …

Run the next cell to save the map config if you want to save the style you changed on the map

In [15]:
with open("./map_configs/hello_overture_map_1.json", "w") as f:
    f.write(json.dumps(map_1.config))